In [ ]:
import csv
import os
import regex as re

from collections import namedtuple
from pprint import pprint


In [ ]:
def resolve_filename(filename):
    if filename.startswith("~"):
        return os.path.expanduser(filename)
    else:
        return filename
    
def with_resolution(function):
    def resolution_function(filename, *args, **kwargs):
        resolved_name = resolve_filename(filename)
        return function(resolved_name, *args, **kwargs)
    return resolution_function

def get_real_path(filename):
    return os.path.realpath(filename)

def with_real_path(function):
    def wrapped_function(filename, *args, **kwargs):
        real_path = os.path.realpath(filename)
        return function(real_path, *args, **kwargs)
    return wrapped_function

# def resolve_link(filename):
#     if os.path.islink(filename):
#         return os.readlink(filename)
#     else:
#         return filename

@with_resolution
def read_all_lines(filename, delete_trailing_carriage_returns=True):
    with open(filename, mode="rt") as fp:
        if delete_trailing_carriage_returns:
            return [line[:-1] for line in  fp.readlines()]
        else:
            return fp.readlines()
        
def remove_empty_and_whitespace_lines(lines):
    return [line for line in lines if len(line.strip()) > 0]

def write_csv(filename, fieldnames, rows):
    with open(filename, mode="w", newline="") as fp:
        writer = csv.DictWriter(fp, fieldnames)
        writer.writeheader()
        writer.writerows(rows)
        
@with_resolution
def change_ext(filename, new_ext):
    filename_wo_ext, _ = os.path.splitext(filename)
    if new_ext[0] == ".":
        return f"{filename_wo_ext}{new_ext}"
    else:
        return f"{filename_wo_ext}.{new_ext}"

def list_files(directory=".", filter=lambda f:f):
    for f in sorted(os.listdir(directory), key=lambda s : s.lower()):
        if os.path.isfile(f) and filter(f):
            print(f)

In [ ]:
write_csv(change_ext(TO_BE_PARSED, "csv"), ParsedFile._fields, [row._asdict() for row in rows])

In [ ]:
def is_icloud_file(filename):
    return filename.startswith(".") and filename.endswith(".icloud")

def get_name_for_icloud_file(filename):
    return filename[1:-7]

def to_lower(s):
    return s.lower()

def sort_ci(string_list):
    return sorted(string_list, sort=to_lower)

@with_real_path
def get_files(folder):
    return [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]


files = get_files(os.path.realpath(os.path.expanduser("~/Downloads")))
# files = [f for f in os.listdir(os.readlink(os.path.expanduser("~/Downloads"))) if os.path.isfile(f)]
icloud_files = sorted([get_name_for_icloud_file(f) for f in files if is_icloud_file(f)])
#icloud_files = [get_name_for_icloud_file(f) for f in files if f.startswith(".") and f.endswith(".icloud")]
local_files = sorted([f for f in files if not is_icloud_file(f)])

#pprint(icloud_files)
#pprint(local_files)
pprint([f for f in icloud_files if os.path.splitext(f)[1].lower() in [".pdf", ".mobi", ".epub"]])

In [ ]:
os.path.realpath(os.path.expanduser("~/Downloads"))